In [38]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime

yf.pdr_override()

In [53]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

In [57]:
data = pdr.get_data_yahoo(
    "ADANIENT.NS", end=datetime.today(), session=session
).reset_index()
data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-01,1.364148,1.401700,1.334259,1.362615,-0.010452,1080397
1,2002-07-02,1.386372,1.390971,1.356484,1.373344,-0.010535,1016147
2,2002-07-03,1.371811,1.387139,1.364914,1.373344,-0.010535,980394
3,2002-07-04,1.379475,1.401700,1.379475,1.381008,-0.010593,972747
4,2002-07-05,1.388672,1.388672,1.372578,1.377942,-0.010570,974496
...,...,...,...,...,...,...,...
5356,2024-01-19,2945.350098,2949.449951,2903.050049,2915.649902,2915.649902,2014096
5357,2024-01-22,2915.649902,2915.649902,2915.649902,2915.649902,2915.649902,0
5358,2024-01-23,3014.949951,3014.949951,2885.500000,2896.550049,2896.550049,1827002
5359,2024-01-24,2901.000000,2929.100098,2871.250000,2903.449951,2903.449951,5807940


In [49]:
yf.Ticker("ADANIENT.NS").info

{'address1': 'Adani Corporate House',
 'address2': 'Shantigram Near Vaishnodevi Circle S. G. Highway, Khodiyar',
 'city': 'Ahmedabad',
 'zip': '382421',
 'country': 'India',
 'phone': '91 79 2555 5286',
 'fax': '91 79 2555 5500',
 'website': 'https://www.adanienterprises.com',
 'industry': 'Thermal Coal',
 'industryKey': 'thermal-coal',
 'industryDisp': 'Thermal Coal',
 'sector': 'Energy',
 'sectorKey': 'energy',
 'sectorDisp': 'Energy',
 'longBusinessSummary': "Adani Enterprises Limited, together with its subsidiaries, engages in the coal trading, mining, logistics, power generation, edible oil and agro commodities, and other businesses in India and internationally. The company trades in coal; offers integrated coal management services; imports apples, pears, kiwis, oranges, grapes, and other fruits; markets fruits under the FARM-PIK brand; generates solar and wind energy; and manufactures solar panels. It is also involved in the mining of various coal and iron ore properties, as well

In [66]:
datetime.strptime("2015-01-01", "%Y-%m-%d")

datetime.datetime(2015, 1, 1, 0, 0)